In [2]:
from pandas import *
from sparql_dataframe import get

endpoint = 'https://dati.cultura.gov.it/sparql'

endpointDB = 'https://dbpedia.org/sparql'

In [9]:
#since I have the institute for the properties I should keep the site instead of the subject
query = '''
SELECT DISTINCT(?urlcity) COUNT(DISTINCT(?s) as ?count) ?urlcitylabel ?dbpedia WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?urlcitylabel.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 rdfs:label ?label.
?urlcity2 owl:sameAs ?dbpedia.
FILTER (contains(str(?dbpedia), "dbpedia"))
}
'''

dbPediaLinks = get(endpoint, query)
dbPediaLinks

,urlcity,callret-1,urlcitylabel,dbpedia
0,http://dati.beniculturali.it/mibact/eventi/res...,1,Lonato del Garda,http://dbpedia.org/resource/Lonato_del_Garda
1,http://dati.beniculturali.it/mibact/eventi/res...,14,La Maddalena,http://dbpedia.org/resource/La_Maddalena
2,http://dati.beniculturali.it/mibact/eventi/res...,15,Cividate Camuno,http://dbpedia.org/resource/Cividate_Camuno
3,http://dati.beniculturali.it/mibact/eventi/res...,16,Russi,http://dbpedia.org/resource/Russi
4,http://dati.beniculturali.it/mibact/eventi/res...,256,Sassari,http://dbpedia.org/resource/Sassari
...,...,...,...,...
789,http://dati.beniculturali.it/mibact/eventi/res...,4,Giovinazzo,http://dbpedia.org/resource/Giovinazzo
790,http://dati.beniculturali.it/mibact/eventi/res...,2,Siracusa,"http://dbpedia.org/resource/Syracuse,_Sicily"
791,http://dati.beniculturali.it/mibact/eventi/res...,13,Lecco,http://dbpedia.org/resource/Lecco
792,http://dati.beniculturali.it/mibact/eventi/res...,16,Canino,http://dbpedia.org/resource/Canino


In [10]:
dbpedia = dbPediaLinks["dbpedia"].drop_duplicates()
link = []


Regions = DataFrame({})

for idx, item in dbpedia.items():
    query = '''
        SELECT ('''f'<{item}>'''') as ?dbpedia ?region ?regionLabel WHERE{
        '''f'<{item}>'''' dbo:region ?region.
        ?region rdfs:label ?regionLabel.
        FILTER (langMatches(lang(?regionLabel), "it"))
    }'''
    coordinates = get(endpointDB, query)
    link.append(item)
    Regions = concat([Regions, coordinates])
    
Regions

,dbpedia,region,regionLabel
0,http://dbpedia.org/resource/La_Maddalena,http://dbpedia.org/resource/Sardinia,Sardegna (isola)
1,http://dbpedia.org/resource/La_Maddalena,http://dbpedia.org/resource/Sardinia,Sardegna
2,http://dbpedia.org/resource/La_Maddalena,http://dbpedia.org/resource/Sardinia,Regno di Sardegna
0,http://dbpedia.org/resource/Cividate_Camuno,http://dbpedia.org/resource/Lombardy,Lombardia
1,http://dbpedia.org/resource/Cividate_Camuno,http://dbpedia.org/resource/Lombardy,Consiglio regionale della Lombardia
...,...,...,...
1,http://dbpedia.org/resource/Lecco,http://dbpedia.org/resource/Lombardy,Consiglio regionale della Lombardia
0,http://dbpedia.org/resource/Canino,http://dbpedia.org/resource/Lazio,Latium
1,http://dbpedia.org/resource/Canino,http://dbpedia.org/resource/Lazio,Lazio
0,http://dbpedia.org/resource/Terracina,http://dbpedia.org/resource/Lazio,Latium


In [18]:
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Marken", value='Marche')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Consiglio regionale della Lombardia", value='Lombardia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Consiglio regionale del Friuli-Venezia Giulia", value='Friuli-Venezia Giulia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Friuli", value='Friuli-Venezia Giulia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Emilia", value='Emilia-Romagna')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Emilia Romagna", value='Emilia-Romagna')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Sardegna (isola)", value='Sardegna')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Regno di Sardegna", value='Sardegna')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Isola di Sicilia", value='Sicilia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Sicilia (provincia romana)", value='Sicilia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Consiglio regionale della Calabria", value='Calabria')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Aosta", value="Valle d'Aosta")
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Provincia di Udine", value='Friuli-Venezia Giulia')
Regions["regionLabel"] = Regions["regionLabel"].replace(to_replace="Latium", value='Lazio')
Regions["regionLabel"].drop_duplicates()
Regions

,dbpedia,region,regionLabel
0,http://dbpedia.org/resource/La_Maddalena,http://dbpedia.org/resource/Sardinia,Sardegna
1,http://dbpedia.org/resource/La_Maddalena,http://dbpedia.org/resource/Sardinia,Sardegna
2,http://dbpedia.org/resource/La_Maddalena,http://dbpedia.org/resource/Sardinia,Sardegna
0,http://dbpedia.org/resource/Cividate_Camuno,http://dbpedia.org/resource/Lombardy,Lombardia
1,http://dbpedia.org/resource/Cividate_Camuno,http://dbpedia.org/resource/Lombardy,Lombardia
...,...,...,...
1,http://dbpedia.org/resource/Lecco,http://dbpedia.org/resource/Lombardy,Lombardia
0,http://dbpedia.org/resource/Canino,http://dbpedia.org/resource/Lazio,Lazio
1,http://dbpedia.org/resource/Canino,http://dbpedia.org/resource/Lazio,Lazio
0,http://dbpedia.org/resource/Terracina,http://dbpedia.org/resource/Lazio,Lazio


In [22]:
AllData = dbPediaLinks.merge(Regions, left_on="dbpedia", right_on="dbpedia")[["callret-1", "urlcitylabel", "regionLabel"]]
AllData.drop_duplicates()

,callret-1,urlcitylabel,regionLabel
0,14,La Maddalena,Sardegna
3,15,Cividate Camuno,Lombardia
5,16,Russi,Emilia-Romagna
7,256,Sassari,Sardegna
10,12,Ruvo di Puglia,Puglia
...,...,...,...
1160,4,Giovinazzo,Puglia
1161,2,Siracusa,Sicilia
1164,13,Lecco,Lombardia
1166,16,Canino,Lazio


In [25]:
for idx, row in AllData.iterrows():
    print(row["regionLabel"])

Sardegna
Sardegna
Sardegna
Lombardia
Lombardia
Emilia-Romagna
Emilia-Romagna
Sardegna
Sardegna
Sardegna
Puglia
Piemonte
Toscana
Trentino-Alto Adige
Lazio
Lazio
Campania
Sardegna
Sardegna
Sardegna
Toscana
Liguria
Liguri
Sardegna
Sardegna
Sardegna
Calabria
Calabria
Puglia
Molise
Toscana
Puglia
Molise
Veneto
Lombardia
Lombardia
Toscana
Liguria
Liguri
Sardegna
Sardegna
Sardegna
Campania
Campania
Lazio
Lazio
Abruzzo
Sardegna
Sardegna
Sardegna
Lazio
Lazio
Piemonte
Toscana
Emilia-Romagna
Emilia-Romagna
Marche
Marche
Liguria
Liguri
Toscana
Veneto
Abruzzo
Lombardia
Lombardia
Lombardia
Lombardia
Piemonte
Lazio
Lazio
Toscana
Emilia-Romagna
Emilia-Romagna
Emilia-Romagna
Emilia-Romagna
Marche
Marche
Friuli-Venezia Giulia
Friuli-Venezia Giulia
Friuli-Venezia Giulia
Friuli-Venezia Giulia
Emilia-Romagna
Emilia-Romagna
Abruzzo
Marche
Marche
Piemonte
Toscana
Piemonte
Liguria
Liguri
Lombardia
Lombardia
Marche
Marche
Lombardia
Lombardia
Lazio
Lazio
Lazio
Lazio
Calabria
Calabria
Sardegna
Sardegna
Sardegna


In [32]:
import json
EventCount = {}
for idx, row in AllData.iterrows():
    if row["regionLabel"] not in EventCount:
        EventCount[row["regionLabel"]] = {}
        EventCount[row["regionLabel"]][row["urlcitylabel"]] = row["callret-1"]
    else: 
        if row["urlcitylabel"] not in EventCount:
            EventCount[row["regionLabel"]][row["urlcitylabel"]] = row["callret-1"]
        else: 
            EventCount[row["regionLabel"]][row["urlcitylabel"]] += row["callret-1"]

with open("RegionCityEventCount.json", "w", encoding="utf-8") as f:
    json.dump(EventCount, f, indent=4, ensure_ascii=False)